Task 1: Natural Language Understanding (NLU)

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def process_input(user_input):
    doc = nlp(user_input)

    # Extracting entities (e.g., dates, locations)
    for ent in doc.ents:
        print(f"{ent.text} - {ent.label_}")

    # Simple rule-based intent detection
    if "book a flight" in user_input.lower():
        return "Booking Flight"
    elif "hotel" in user_input.lower():
        return "Hotel Inquiry"
    return "General Inquiry"

# Example usage
user_input = "I want to book a flight to New York on September 21"
intent = process_input(user_input)
print(f"Detected Intent: {intent}")


New York - GPE
September 21 - DATE
Detected Intent: Booking Flight


Task 2: Training on Travel-Related Conversations

Using Hugging Face's transformers library for training on travel-related conversations.

In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install transformers

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Load the pre-trained BlenderBot model
model_name = "facebook/blenderbot-400M-distill"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the text generation pipeline
chatbot_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def chatbot_response(user_input):
    # Tokenize the user input
    inputs = tokenizer.encode(user_input, return_tensors="pt")

    # Generate a response from the model
    response_ids = model.generate(inputs, max_length=100, num_return_sequences=1)

    # Decode the generated response back to text
    response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return response

# Example conversation
user_input = "I'm looking for a flight to Paris next week"
response = chatbot_response(user_input)
print(f"Chatbot: {response}")


The model 'BlenderbotForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'Mi

Chatbot:  I hope you have a great time! I've always wanted to visit Paris. Have you been before?


Task 3: Database Integration and Querying

Install SQLite in Colab to manage a small travel information database.

In [ ]:
import sqlite3

# Connect to the travel database
conn = sqlite3.connect('travel_info.db')
c = conn.cursor()

# Create a table for flight schedules
c.execute('''CREATE TABLE IF NOT EXISTS flights
             (flight_number TEXT, destination TEXT, date TEXT, time TEXT)''')

# Insert some dummy flight data
c.execute("INSERT INTO flights VALUES ('AA123', 'New York', '2024-09-21', '08:30')")

conn.commit()

# Query flight information
def query_flight(destination, date):
    c.execute("SELECT * FROM flights WHERE destination=? AND date=?", (destination, date))
    result = c.fetchone()
    if result:
        print(f"Flight: {result}")
    else:
        print("No flights found.")

query_flight('New York', '2024-09-21')


Flight: ('AA123', 'New York', '2024-09-21', '08:30')


Task 4: API Handling

Use the requests library to handle API calls. For demonstration, here’s an example of querying a travel API (Skyscanner or any other travel service API).

In [ ]:
!pip install requests


In [ ]:
import requests

# Example API request (replace with actual API and keys)
api_url = "https://test.api.amadeus.com"
headers = {"3lO1vlAw913kR45Wh2hzCLXV6ieAdaXg": "zgg7pEJtCGZpywtC"}

response = requests.get(api_url, headers=headers)

if response.status_code == 200:
    data = response.json()
    print(data)  # Parse and use the flight data
else:
    print("API requesting...")


API requesting...


In [ ]:
import requests

# Replace these with your Amadeus API credentials
API_KEY = "3lO1vlAw913kR45Wh2hzCLXV6ieAdaXg"
API_SECRET = "zgg7pEJtCGZpywtC"

# Amadeus authentication URL
auth_url = "https://test.api.amadeus.com/v1/security/oauth2/token"

# Request body for OAuth2 token
data = {
    'grant_type': 'client_credentials',
    'client_id': API_KEY,
    'client_secret': API_SECRET
}

# Making the request
response = requests.post(auth_url, data=data)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access Token:", access_token)
else:
    print("Error:", response.status_code, response.text)


Access Token: 3kUAtRI4AodnkvhG34y89ANgjmvE


In [ ]:
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Example API call to Amadeus
flight_url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
params = {
    "originLocationCode": "LAX",
    "destinationLocationCode": "JFK",
    "departureDate": "2024-10-15",
    "adults": 1
}

response = requests.get(flight_url, headers=headers, params=params)

if response.status_code == 200:
    print(response.json())
else:
    print("Error:", response.status_code, response.text)


{'meta': {'count': 33, 'links': {'self': 'https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=LAX&destinationLocationCode=JFK&departureDate=2024-10-15&adults=1'}}, 'data': [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2024-09-21', 'lastTicketingDateTime': '2024-09-21', 'numberOfBookableSeats': 7, 'itineraries': [{'duration': 'PT5H11M', 'segments': [{'departure': {'iataCode': 'ONT', 'terminal': '2', 'at': '2024-10-15T23:49:00'}, 'arrival': {'iataCode': 'JFK', 'terminal': '5', 'at': '2024-10-16T08:00:00'}, 'carrierCode': 'B6', 'number': '1654', 'aircraft': {'code': '321'}, 'operating': {'carrierCode': 'B6'}, 'duration': 'PT5H11M', 'id': '11', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '121.20', 'base': '100.00', 'fees': [{'amount': '0.00', 'type': 'SUPPLIER'}, {'amount': '0.00', 'type': 'TICKE

In [ ]:
def get_flight_offers_mock(origin, destination, departure_date):
    return {
        "flights": [
            {"flight": "LA123", "price": "$200", "departure_time": "10:00 AM", "arrival_time": "2:00 PM"},
            {"flight": "LA456", "price": "$220", "departure_time": "1:00 PM", "arrival_time": "5:00 PM"}
        ]
    }


Task 5: Personalized Travel Recommendations

You can implement simple logic to recommend travel destinations based on preferences.

In [ ]:
def recommend_destination(user_preferences):
    if 'beach' in user_preferences:
        return 'How about visiting Hawaii or Bali?'
    elif 'adventure' in user_preferences:
        return 'You might enjoy hiking in the Swiss Alps!'
    else:
        return 'I recommend visiting Paris for a cultural experience.'

# Example user preferences
user_prefs = ['beach', 'luxury']
print(recommend_destination(user_prefs))


How about visiting Hawaii or Bali?


Task 6: Booking and Cancellation Management

You can simulate booking and cancellation management, either via database operations or API calls.

In [ ]:
def book_flight(flight_number, user_details):
    # Perform booking logic here (using API or database)
    print(f"Booking flight {flight_number} for {user_details['name']}")
    return "Booking Confirmed"

def cancel_booking(booking_id):
    # Perform cancellation logic here
    print(f"Cancelling booking {booking_id}")
    return "Booking Cancelled"

# Example booking
user_details = {'name': 'John Doe', 'email': 'john@example.com'}
print(book_flight('AA123', user_details))
print(cancel_booking('ABC123'))


Booking flight AA123 for John Doe
Booking Confirmed
Cancelling booking ABC123
Booking Cancelled


Task 7: Real-Time Travel Updates and Customer Support

Simulate real-time updates using predefined messages and offer basic customer support using a rule-based approach.

In [ ]:
def provide_real_time_update(flight_number):
    updates = {
        'AA123': 'Your flight is delayed by 30 minutes.',
        'BA456': 'Your flight is on time.'
    }
    return updates.get(flight_number, "No updates available")

# Example usage
print(provide_real_time_update('AA123'))

def customer_support(query):
    faqs = {
        'change flight': 'You can change your flight by visiting the "Manage Booking" section.',
        'cancel flight': 'You can cancel your flight by contacting customer service.'
    }
    return faqs.get(query.lower(), 'Please contact our support for further assistance.')

# Example support query
print(customer_support('change flight'))


Your flight is delayed by 30 minutes.
You can change your flight by visiting the "Manage Booking" section.
